# Cleaning & Table Formatting

---

In this notebook I will be cleaning and editing and formating some data used in the rest of the analysis

In [2]:
import pandas as pd
poke = pd.read_csv('./Data/pokedex_formal.csv')

In [3]:
# Creating a common type combo for coloring purposes.  
poke.head()

,PokedexNumber,Name,Type,Total,HP,Attack,Defense,SpecialAttack,SpecialDefense,Speed,Type_P,Type_S,Mega,legendary
0,150,MewtwoMega Mewtwo X,"['Psychic', 'Fighting']",800,126,190,100,154,100,130,Psychic,Fighting,1,1
1,150,MewtwoMega Mewtwo Y,['Psychic'],780,106,150,70,194,120,140,Psychic,Psychic,1,1
2,384,RayquazaMega Rayquaza,"['Dragon', 'Flying']",780,105,180,100,180,100,115,Dragon,Flying,1,1
3,382,KyogrePrimal Kyogre,['Water'],770,100,150,90,180,160,90,Water,Water,0,1
4,383,GroudonPrimal Groudon,"['Ground', 'Fire']",770,100,180,160,150,90,90,Ground,Fire,0,1


In [4]:
# Issues when i insitially read this in that needed to be cleaned up
def type_clean(types):
    chars = chars = ["'",'[',']',' ']
    cleaned_type = ''.join(letter for letter in types if letter not in chars)
    try:
        cleaned_list = cleaned_type.split(',')
        return str(sorted(cleaned_list))
    except:
        return cleaned_type

In [5]:
poke['Type'] = poke['Type'].apply(type_clean)
poke.head()

,PokedexNumber,Name,Type,Total,HP,Attack,Defense,SpecialAttack,SpecialDefense,Speed,Type_P,Type_S,Mega,legendary
0,150,MewtwoMega Mewtwo X,"['Fighting', 'Psychic']",800,126,190,100,154,100,130,Psychic,Fighting,1,1
1,150,MewtwoMega Mewtwo Y,['Psychic'],780,106,150,70,194,120,140,Psychic,Psychic,1,1
2,384,RayquazaMega Rayquaza,"['Dragon', 'Flying']",780,105,180,100,180,100,115,Dragon,Flying,1,1
3,382,KyogrePrimal Kyogre,['Water'],770,100,150,90,180,160,90,Water,Water,0,1
4,383,GroudonPrimal Groudon,"['Fire', 'Ground']",770,100,180,160,150,90,90,Ground,Fire,0,1


In [7]:
# Value counts of current typings and type-sets
poke['Type'].value_counts()


['Normal']                61
['Water']                 59
['Psychic']               37
['Grass']                 33
['Fire']                  28
['Electric']              27
['Flying', 'Normal']      24
['Fighting']              20
['Bug']                   17
['Fairy']                 15
['Poison']                15
['Grass', 'Poison']       15
['Bug', 'Flying']         14
['Bug', 'Poison']         13
['Ice']                   13
['Ground']                13
['Dragon']                11
['Ground', 'Water']       10
['Rock', 'Water']         10
['Ghost']                 10
['Dark']                  10
['Rock']                   9
['Ground', 'Rock']         9
['Dragon', 'Flying']       8
['Psychic', 'Steel']       7
['Dragon', 'Ground']       7
['Fighting', 'Fire']       7
['Flying', 'Water']        7
['Flying', 'Psychic']      6
['Bug', 'Steel']           6
                          ..
['Psychic', 'Rock']        2
['Flying', 'Ice']          2
['Ghost', 'Water']         2
['Bug', 'Fire'

In [8]:
poke['Type_P'].value_counts()+poke['Type_S'].value_counts()

Bug          87
Dark         60
Dragon       61
Electric     71
Fairy        55
Fighting     73
Fire         90
Flying      101
Ghost        49
Grass       121
Ground       79
Ice          50
Normal      163
Poison       77
Psychic     124
Rock         67
Steel        52
Water       184
dtype: int64

In [9]:
type_count_df = pd.concat([pd.DataFrame(poke['Type_P'].value_counts()),pd.DataFrame(poke['Type_S'].value_counts())],
          axis=1)
type_count_df['Total'] = type_count_df['Type_P']+type_count_df['Type_S'] 
type_count_df.reset_index(inplace=True)

In [10]:
#!pip install pandasql
import pandasql
from pandasql import sqldf

In [11]:
type_sum = pd.DataFrame(columns = ['Type','Total','HP','Attack','Defense','SpecialAttack','SpecialDefense','Speed','Count'])
type_sum

,Type,Total,HP,Attack,Defense,SpecialAttack,SpecialDefense,Speed,Count


In [12]:
# Using a sql qeury to create sums and averages of individual types 
# 
for a_type in type_count_df['index']:
    query = '''
SELECT Name, Total, HP, Attack, Defense, SpecialAttack, SpecialDefense, Speed
FROM poke
WHERE Type_P ="{}" OR Type_S ="{}";
'''.format(a_type, a_type)
    a_type_df = sqldf(query, globals())
    a_type_mean = a_type_df.describe().loc['mean']
    
    type_sum.loc[len(type_sum)]=[a_type, a_type_mean[0],a_type_mean[1],a_type_mean[2],a_type_mean[3],
                               a_type_mean[4],a_type_mean[5],a_type_mean[6],int(a_type_df.describe().loc['count'][0])]

In [13]:
int(a_type_df.describe().loc['count'][0])

125

In [14]:
type_sum

,Type,Total,HP,Attack,Defense,SpecialAttack,SpecialDefense,Speed,Count
0,Bug,378.257143,56.642857,71.700000,70.285714,53.271429,63.942857,62.414286,70
1,Dark,456.520000,70.040000,95.520000,71.160000,75.600000,68.440000,75.760000,50
2,Dragon,541.760000,82.900000,105.760000,86.620000,97.440000,86.900000,82.140000,50
3,Electric,433.227273,64.340909,69.227273,61.863636,85.863636,69.750000,82.181818,44
4,Fairy,415.925000,68.475000,61.575000,70.400000,76.500000,83.125000,55.850000,40
5,Fighting,470.490566,74.943396,104.660377,74.056604,66.924528,73.377358,76.528302,53
6,Fire,465.080645,69.919355,85.290323,68.935484,92.048387,73.306452,75.580645,62
7,Flying,451.888889,71.505051,80.131313,67.818182,75.383838,70.808081,86.242424,99
8,Ghost,438.179487,63.025641,77.205128,77.538462,82.256410,76.923077,61.230769,39
9,Grass,417.000000,66.511364,73.136364,71.318182,74.068182,71.545455,60.420455,88


In [15]:
type_summary_df = pd.merge(left = type_count_df, right= type_sum, left_on = 'index', right_on = 'Type' )
type_summary_df.rename(columns = {'Total_x':'Occurances'}, inplace=True)

In [16]:
type_summary_df.rename(columns = {'index':'Type','Type_P':'Primary_Type_Occ','Type_S':'Secondary_Type_Occ','Occurances':'Total_Occ' }, inplace=True)
type_summary_df.to_csv('./Data/Pokemon_Summary.csv')

In [18]:
subtype_df.describe().loc['mean']

Total             428.368
HP                 71.192
Attack             73.776
Defense            74.032
SpecialAttack      74.536
SpecialDefense     70.016
Speed              64.816
Name: mean, dtype: float64